## Introducion
In this project, stacks (lifo method) will be used to implement an algorithm that can evaluate numerical expressions.

Calculating the result of a complex numerical expression isn't something that a computer processor does right out of the box. Behind the scenes, Python uses an algorithm to evaluate this expression.

The goal of this project is to use the stack data structure to implement an algorithm that can evaluate complex numerical expressions.

In [1]:
## importing LinkedList class from file

from linked_list import LinkedList

## Implementing Stack class

class Stack(LinkedList):
    
    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data

    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

### Implementing the tokenize function

In [2]:
def tokenize(some_string):
    return some_string.split()

# test
print(tokenize("12 2 4 + / 21 *")) # postfix vs infix notation (computer uses postfix notation)   

['12', '2', '4', '+', '/', '21', '*']


### Functions to process operators in postfix evaluation with use of a stack

It is very important to perform the operation between the elements that was second and the top elements. Doing it the other way around will give it a wrong result.

In [3]:
## subtraction
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)

## addition    
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result)

## multiplication
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result)

## division    
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)

## to the power of
def process_power(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

Creating a function to evaluate postfix expressions
The following steps needs to be followed to implement the evaluate_postfix() function.

- Initialize an empty stack.
- Tokenize the expression using the tokenize() function.
- For each token, do:
    - If the token an operator, call the corresponding function to process it. For example, if we find a + we call the process_plus() function.
    - Otherwise (the token is a number) and we push that number to the top of the stack. Since each token is a string, we'll need to convert it to a float first.
- Return the value that is left in the stack.

In [4]:
def evaluate_postfix(expression):
    tokenized = tokenize(expression)
    stack = Stack() # it is important that an expression should have two numbers before any operator due to postfix notation
    
    for token in tokenized:
        if token == "+":
            process_plus(stack)
        elif token == "-":
            process_minus(stack)
        elif token == "*":
            process_times(stack)
        elif token == "/":
            process_divide(stack)
        elif token == "**":
            process_power(stack)
        else:
            # when token is not and operator it will be added to the stack
            stack.push(float(token))
    return stack.pop()

### Testing the implementation (make sure to add spaces so that tokenization works correctly)


In [5]:
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for expression in expressions:
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


### Creating a precedence dictionary (for calculation purposes and to change to infix later)

In [6]:
precedence = {
    "+": 1,
    "-": 1,
    "*": 2,
    "/": 2,
    "**": 3
}

## Test
print(precedence["/"] < precedence["-"])
print(precedence["+"] < precedence["*"])
print(precedence["+"] < precedence["-"])
print(precedence["/"] < precedence["**"])

False
True
False
True


### Processing infix to postfix (creating initial functions)

In [7]:
## 1. Push open bracket into stack (will be used once closing bracket arrives into the stack as well)
def process_opening_parenthesis(stack):
    stack.push("(")
    
## 2. Push closing bracket to the stack 
def process_closing_parenthesis(stack, postfix):
    while stack.peek() != "(": # while the top of the stack does not have an opening bracket append it to the postfix list
        postfix.append(stack.pop())
    # Remove the opening bracket
    stack.pop()

## 3. Process precedence of operator (pop to stack)
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        postfix.append(stack.pop())
    stack.push(operator)
    
## 4. Processing numbers
def process_number(postfix, number):
    postfix.append(number)

### Implementing the Shunting-yard Algorithm (making use of previous functions)
- Start by splitting the expression into tokens using the tokenize() function.
- Initialize an empty stack.
- Initialize and empty postfix token list.
- Iterate over all tokens and for each of them:
    - If the token is "(" we call the process_opening_parenthesis() function.
    - If the token is ")" we call the process_closing_parenthesis() function.
    - If the token is an operator we call the process_operator() function.
    - Otherwise, the token is a number and we call the process_number() function.
- After processing all tokens, make use of a while loop to pop the remaining stack elements into the postfix token list.
- Use the str.join() method to convert the postfix token list into a string.

In [8]:
def infix_to_postfix(expression):
    
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    
    for token in tokens:
        if token == "(":
            process_opening_parenthesis(stack)
        elif token == ")":
            process_closing_parenthesis(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
            
    while len(stack) > 0:
        postfix.append(stack.pop())
    
    return " ".join(postfix)

### Evaluating any infix expression

In [9]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

## Testing

expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expression in expressions:
    print(evaluate(expression))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0
